In [ ]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data_dir = './Dataset'

# Load dataset from directory
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,image_size=(150,150),batch_size=32,label_mode="categorical"
)

class_names = dataset.class_names

In [ ]:
# Split dataset into training and testing
train_size = 0.8
train_ds = dataset.take(int(len(dataset) * train_size))
test_ds = dataset.skip(int(len(dataset) * train_size))

In [ ]:
# Split the train dataset further into train and validation
val_size = int(0.2 *  len(train_ds))
val_ds = train_ds.take(val_size)
train_ds = train_ds.skip(val_size)

In [ ]:
# Add this data augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.1)
])

# Apply data augmentation to the dataset
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
# Prefetch for performance
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def dataset_to_numpy(dataset):
    images, labels = [], []
    for batch_images, batch_labels in dataset:
        images.append(batch_images.numpy())
        labels.append(batch_labels.numpy())
    return np.concatenate(images), np.concatenate(labels)

# Extract training and testing data
x_train, y_train = dataset_to_numpy(train_ds)
x_test, y_test = dataset_to_numpy(test_ds)

x_train=x_train/255.0
x_test=x_test/255.0

In [ ]:
model = Sequential([
    # Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Fully Connected Layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(17, activation='softmax')
])

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy']
)

In [ ]:
history = model.fit(x_train, y_train, epochs=25, batch_size=64, validation_data=val_ds)
model.save('trained.keras')

In [ ]:
# Plot the training and validation accuracy/loss
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
# Pick a random image from the test set for prediction
idx = random.randint(0, len(x_test) - 1)
test_image = x_test[idx]
true_class_idx = np.argmax(y_test[idx])  # True class index
true_class_name = class_names[true_class_idx]

plt.imshow(test_image)
plt.axis('off')  # Hide axis for a cleaner view
# plt.title(f"Actual: {true_class_name}")
plt.show()

# Predict the class for the test image
predicted_prob = model.predict(test_image.reshape(1, 150,150, 3))  # Reshape the image
predicted_class_idx = np.argmax(predicted_prob)  # Get the predicted class index
predicted_class_name = class_names[predicted_class_idx]

# Display predicted and actual class names
print(f"Predicted class: {predicted_class_name}")
print(f"Actual class: {true_class_name}")